In [1]:
import numpy as np
import cv2 #video capturing
import math
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing import image
from keras.utils import np_utils
from skimage.transform import resize
from keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
count = 0
videoFile = "Tom and jerry.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5)

#x=1
def get_frames():
	count = 0
	videoFile = "Tom and jerry.mp4"
	cap = cv2.VideoCapture(videoFile)
	frameRate = cap.get(5)
	while(cap.isOpened()):
		frameId = cap.get(1) #current frame number
		ret, frame = cap.read()
		if (ret != True):
			break
		if (frameId % math.floor(frameRate) == 0):
			filename = "frame%d.jpg" % count;count+=1
			cv2.imwrite(filename,frame)
	cap.release()
	print("Done!")


data = pd.read_csv('mapping.csv')
X = []
for img_name in data.Image_ID:
	img = plt.imread(''+img_name)
	X.append(img)
X = np.array(X)

y = data.Class
dummy_y = np_utils.to_categorical(y)

image = []
for i in range(0,X.shape[0]):
	a = resize(X[i], preserve_range=True,output_shape=(224,224)).astype(int)
	image.append(a)
X = np.array(image)


X = preprocess_input(X,mode='tf')


C:\Users\maxfa\Anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\maxfa\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y,test_size=0.3,random_state=42)


In [8]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer,Dropout

In [10]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58892288/58889256 [==============================] - 15s 0us/step


In [11]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape,X_valid.shape

((208, 7, 7, 512), (90, 7, 7, 512))

In [13]:
X_train = X_train.reshape(208,7*7*512)
X_valid = X_valid.reshape(90,7*7*512)
train = X_train/X_train.max()
X_valid = X_valid/X_train.max()

In [17]:
model = Sequential()
model.add(InputLayer((7*7*512,)))
model.add(Dense(units=1024,activation='sigmoid'))
model.add(Dense(3,activation='softmax'))

In [18]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3075      
Total params: 25,694,211
Trainable params: 25,694,211
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [21]:
model.fit(train,y_train,epochs=100,validation_data=(X_valid,y_valid))

Train on 208 samples, validate on 90 samples
Epoch 1/100
208/208 [==============================] - 4s 18ms/step - loss: 0.9960 - accuracy: 0.5433 - val_loss: 0.6317 - val_accuracy: 0.6778
Epoch 2/100
208/208 [==============================] - 3s 16ms/step - loss: 0.3494 - accuracy: 0.9038 - val_loss: 0.4998 - val_accuracy: 0.7889
Epoch 3/100
208/208 [==============================] - 3s 17ms/step - loss: 0.1660 - accuracy: 0.9471 - val_loss: 0.3716 - val_accuracy: 0.8444
Epoch 4/100
208/208 [==============================] - 4s 19ms/step - loss: 0.1024 - accuracy: 0.9808 - val_loss: 0.3577 - val_accuracy: 0.8556
Epoch 5/100
208/208 [==============================] - 3s 17ms/step - loss: 0.0561 - accuracy: 0.9952 - val_loss: 0.3364 - val_accuracy: 0.8444
Epoch 6/100
208/208 [==============================] - 3s 16ms/step - loss: 0.0400 - accuracy: 0.9952 - val_loss: 0.3416 - val_accuracy: 0.8556
Epoch 7/100
208/208 [==============================] - 3s 16ms/step - loss: 0.0367 - accura

208/208 [==============================] - 3s 17ms/step - loss: 0.0156 - accuracy: 0.9952 - val_loss: 0.3732 - val_accuracy: 0.8667
Epoch 58/100
208/208 [==============================] - 3s 16ms/step - loss: 0.0246 - accuracy: 0.9904 - val_loss: 0.3282 - val_accuracy: 0.8667
Epoch 59/100
208/208 [==============================] - 3s 17ms/step - loss: 0.0134 - accuracy: 0.9952 - val_loss: 0.4951 - val_accuracy: 0.8444
Epoch 60/100
208/208 [==============================] - 4s 17ms/step - loss: 0.0134 - accuracy: 0.9952 - val_loss: 0.3596 - val_accuracy: 0.8556
Epoch 61/100
208/208 [==============================] - 3s 16ms/step - loss: 0.0140 - accuracy: 0.9952 - val_loss: 0.3375 - val_accuracy: 0.8667
Epoch 62/100
208/208 [==============================] - 3s 16ms/step - loss: 0.0113 - accuracy: 0.9952 - val_loss: 0.3720 - val_accuracy: 0.8667
Epoch 63/100
208/208 [==============================] - 3s 16ms/step - loss: 0.0080 - accuracy: 0.9952 - val_loss: 0.5480 - val_accuracy: 0.833

In [24]:
count = 0
videoFile = "Tom and Jerry 3.mp4"
cap = cv2.VideoCapture(videoFile)

frameRate = cap.get(5)

x=1

while(cap.isOpened()):
    frameId = cap.get(1)
    ret,frame = cap.read()
    if(ret != True):
        break
    if(frameId % math.floor(frameRate)==0):
        filename = "test%d.jpg"%count;count+=1
        cv2.imwrite(filename,frame)
cap.release()
print("Done")

Done


In [25]:
test = pd.read_csv('test.csv')

In [28]:
test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [29]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i],preserve_range=True,output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

In [30]:
test_image = preprocess_input(test_image,mode='tf')
test_image = base_model.predict(test_image)
test_image = test_image.reshape(186,7*7*512)
test_image = test_image/test_image.max()

In [31]:
predictions = model.predict_classes(test_image)

In [33]:
print("The screen time of JERRY is", predictions[predictions==1].shape[0], "seconds")
print("The screen time of TOM is", predictions[predictions==2].shape[0], "seconds")

The screen time of JERRY is 7 seconds
The screen time of TOM is 134 seconds
